In [1]:
import numpy as np
import os
import argparse
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.io import ascii
import pandas as pd
from pandas.compat import StringIO
import matplotlib.pyplot as plt
import seaborn as sns

### Define a function that returns a flux (in units of maggies) given an AB magnitude

In [2]:
def flux(mag):
    flux = 10.**(mag/(-2.5))
    return flux

### Define a function that returns inverse variance (in units of magnitudes) given a magnitude and an uncertainty on that magnitude

In [3]:
def ivar(mag, unc):
    flux = 10.**(mag/(-2.5))
    func = flux / 1.086 * unc
    ivar = 1 / func**2
    return ivar

# Read umeh table into pandas dataframe

In [4]:
umeh = pd.read_csv('umeh_table.dat', sep=r'\s{1,}', engine='python', index_col=0)

In [5]:
umeh.head()

,fuv_mag,fuv_unc,nuv_mag,nuv_unc,u_mag,u_unc,g_mag,g_unc,r_mag,r_unc,...,w1_mag,w1_unc,w2_mag,w2_unc,w3_mag,w3_unc,w4_mag,w4_unc,z,ebv
Galaxy,,,,,,,,,,,,,,,,,,,,,
J0826,22.430,0.157,20.393,0.039,19.641,0.034,19.429,0.011,19.139,0.014,...,15.395,0.023,15.146,0.062,11.489,0.156,8.534,0.271,0.603,0.037
J0901,22.036,0.119,20.112,0.039,19.518,0.029,19.288,0.012,18.825,0.011,...,15.242,0.022,15.012,0.064,11.677,0.205,8.716,0.352,0.459,0.057
J0905,23.657,0.430,20.918,0.059,19.911,0.040,19.589,0.014,19.584,0.016,...,15.811,0.027,15.616,0.080,11.763,0.166,8.854,0.305,0.712,0.031
J0944,22.572,0.312,21.238,0.102,20.293,0.049,19.801,0.015,19.295,0.015,...,15.505,0.028,15.186,0.078,11.265,0.142,8.488,0.286,0.514,0.026
J1107,22.919,0.354,21.398,0.104,19.971,0.039,19.517,0.013,19.069,0.012,...,15.586,0.028,14.927,0.057,10.774,0.081,8.254,0.205,0.467,0.076


### Convert maggies into flux

In [6]:
umeh_flux = pd.DataFrame({'fuv_f':flux(umeh.fuv_mag), 'fuv_ivar':ivar(umeh.fuv_mag, umeh.fuv_unc),
                          'nuv_f':flux(umeh.nuv_mag), 'nuv_ivar':ivar(umeh.nuv_mag, umeh.nuv_unc),
                          'u_f':flux(umeh.u_mag), 'u_ivar':ivar(umeh.u_mag, umeh.u_unc),
                          'g_f':flux(umeh.g_mag), 'g_ivar':ivar(umeh.g_mag, umeh.g_unc),
                          'r_f':flux(umeh.r_mag), 'r_ivar':ivar(umeh.r_mag, umeh.r_unc),
                          'i_f':flux(umeh.i_mag), 'i_ivar':ivar(umeh.i_mag, umeh.i_unc),
                          'z_f':flux(umeh.z_mag), 'z_ivar':ivar(umeh.z_mag, umeh.z_unc),
                          'w1_f':flux(umeh.w1_mag), 'w1_ivar':ivar(umeh.w1_mag, umeh.w1_unc),
                          'w2_f':flux(umeh.w2_mag), 'w2_ivar':ivar(umeh.w2_mag, umeh.w2_unc),
                          'w3_f':flux(umeh.w3_mag), 'w3_ivar':ivar(umeh.w3_mag, umeh.w3_unc),
                          'w4_f':flux(umeh.w4_mag), 'w4_ivar':ivar(umeh.w4_mag, umeh.w4_unc)}, index=umeh.index.values)

In [7]:
umeh_flux.head()

,fuv_f,fuv_ivar,nuv_f,nuv_ivar,u_f,u_ivar,g_f,g_ivar,r_f,r_ivar,...,z_f,z_ivar,w1_f,w1_ivar,w2_f,w2_ivar,w3_f,w3_ivar,w4_f,w4_ivar
J0826,1.066596e-09,4.205914e+19,6.963058e-09,1.599302e+19,1.391874e-08,5.266254e+18,1.691999e-08,3.404664e+19,2.210039e-08,1.231980e+19,...,2.876074e-08,6.738201e+17,6.950243e-07,4.615343e+15,8.741783e-07,4.014914e+14,0.000025,7.526801e+10,0.000386,1.078751e+08
J0901,1.533204e-09,3.542956e+19,9.019864e-09,9.530824e+18,1.558834e-08,5.771167e+18,1.926637e-08,2.206466e+19,2.951209e-08,1.119114e+19,...,3.735941e-08,7.309738e+17,8.002027e-07,3.805522e+15,9.890084e-07,2.943742e+14,0.000021,6.162429e+10,0.000326,8.940758e+07
J0905,3.445085e-10,5.374310e+19,4.293387e-09,1.838043e+19,1.085425e-08,6.256618e+18,1.460158e-08,2.822303e+19,1.466898e-08,2.141015e+19,...,2.165708e-08,4.988190e+17,4.738054e-07,7.206628e+15,5.670221e-07,5.731650e+14,0.000020,1.101142e+11,0.000287,1.535536e+08
J0944,9.358365e-10,1.383409e+19,3.197422e-09,1.108816e+19,7.634841e-09,8.426893e+18,1.201158e-08,3.633098e+19,1.914256e-08,1.430465e+19,...,2.370282e-08,7.473222e+17,6.280584e-07,3.813674e+15,8.425584e-07,2.730678e+14,0.000031,6.012886e+10,0.000403,8.898721e+07
J1107,6.798295e-10,2.036353e+19,2.759307e-09,1.432165e+19,1.027070e-08,7.350729e+18,1.560271e-08,2.866639e+19,2.357219e-08,1.473999e+19,...,3.299134e-08,1.127551e+18,5.829080e-07,4.427347e+15,1.069547e-06,3.173291e+14,0.000049,7.479794e+10,0.000499,1.125517e+08


In [8]:
umeh_flux.iloc[:,::2]

,fuv_f,nuv_f,u_f,g_f,r_f,i_f,z_f,w1_f,w2_f,w3_f,w4_f
J0826,1.066596e-09,6.963058e-09,1.391874e-08,1.691999e-08,2.210039e-08,2.810606e-08,2.876074e-08,6.950243e-07,8.741783e-07,0.000025,0.000386
J0901,1.533204e-09,9.019864e-09,1.558834e-08,1.926637e-08,2.951209e-08,3.317416e-08,3.735941e-08,8.002027e-07,9.890084e-07,0.000021,0.000326
J0905,3.445085e-10,4.293387e-09,1.085425e-08,1.460158e-08,1.466898e-08,2.276145e-08,2.165708e-08,4.738054e-07,5.670221e-07,0.000020,0.000287
J0944,9.358365e-10,3.197422e-09,7.634841e-09,1.201158e-08,1.914256e-08,2.333458e-08,2.370282e-08,6.280584e-07,8.425584e-07,0.000031,0.000403
J1107,6.798295e-10,2.759307e-09,1.027070e-08,1.560271e-08,2.357219e-08,2.789973e-08,3.299134e-08,5.829080e-07,1.069547e-06,0.000049,0.000499
J1219,8.379151e-10,2.652162e-09,8.717661e-09,1.550244e-08,3.059145e-08,3.872576e-08,4.630203e-08,9.799412e-07,1.336596e-06,0.000055,0.000752
J1341,1.586354e-09,8.798325e-09,1.984266e-08,2.601357e-08,2.841842e-08,3.268888e-08,3.000544e-08,6.415049e-07,8.135795e-07,0.000037,0.000548
J1506,2.769491e-09,1.366470e-08,1.921321e-08,2.236660e-08,2.802851e-08,3.467369e-08,3.637475e-08,7.870458e-07,1.172735e-06,0.000041,0.000847
J1558,3.056328e-09,9.935735e-09,1.804679e-08,2.342071e-08,3.837072e-08,4.242287e-08,5.062908e-08,8.669619e-07,1.470957e-06,0.000052,0.000360
J1613,4.172535e-10,3.535086e-09,7.950602e-09,1.251988e-08,2.664404e-08,3.338874e-08,4.269726e-08,1.144986e-06,1.877587e-06,0.000082,0.001203


In [9]:
wave = np.array([0.1528, 0.2271, 0.3543, 0.477, 0.6231, 0.7625, 0.9134, 3.368, 4.618, 12.082, 22.194])
templates = ['Composite1', 'Composite2', 'Composite3', 'AGN1', 'AGN2', 'AGN3', 'AGN4', 'SFG1', 'SFG2', 'SFG3', 
             'IR_COLOR1', 'IR_COLOR2', 'IR_COLOR3', 'IR_COLOR4', 'IR_COLOR5', 'IR_COLOR6', 'IR_COLOR7', 'IR_COLOR8']
gals = ['J0826', 'J0901', 'J0905', 'J0944', 'J1107', 'J1219', 'J1341', 'J1506', 'J1558', 'J1613', 'J2116', 'J2140']

# Read templates

In [52]:
#Read the template into a data frame. Name columns something convenient.
def read_template(name):
    temps_path = 'C:/Users/Chris/Documents/GitHub/bates_galaxies_lab/hst/Comprehensive_library/'
    temp = pd.read_csv(temps_path+name+'.txt',
                       names=['rest_wavelength','luminosity','DLnu'],
                       skiprows=[0, 1, 2, 3],
                       sep=r'\s{1,}',
                       engine='python')
    return temp

# Function to match a target wavelength's position.
def mask_wave(temp_wavel, target_wave):
    return np.abs(temp_wavel-target_wave) == np.amin(np.abs(temp_wavel-target_wave))

Function to scale a template using the W1 flux as reference - inputs are galaxy name, and name of template. Plots result, returns scaling factor for luminosity of template.

In [67]:
def fit_template(gal_name, template_name):
    z = umeh.loc[gal_name, 'z']
    template = read_template(template_name)
    
    z_temp_wavel = template.rest_wavelength*(1+z)
    gal_fluxes = umeh_flux.loc[gal_name,::2].values
    W1_wavelength = wave[7]
    # Figure out where the template lines up with W1
    mask = mask_wave(z_temp_wavel, W1_wavelength)
    print('maskcheck---',sum(mask),template.luminosity[mask].values[0])
    # Scale template to match value at W1
    factor = gal_fluxes[7]/float(template.luminosity[mask].values[0])
    luminosity = template.luminosity*factor # Scale

    # Plot
    plot = True

    if plot:
        title = gal_name+'-'+template_name
        fig = plt.figure(figsize=(17,8))
        ax = fig.add_subplot()
        g = sns.lineplot(x=z_temp_wavel, y=luminosity, ax=ax, color='green', alpha=0.8)
        h = sns.scatterplot(x=wave, y=gal_fluxes, ax=ax)
        ax.set_ylim([1e-10,1e-3])
        ax.set_xlim([0.1, 1000.])
        ax.loglog()
        ax.set_title(title)
        plt.savefig(title+'.png')
        plt.clf()
        plt.close()
    return(factor)

In [68]:
# Evaluate
def chi_squared(gal_name, template, factor):
    # Set up variables
    z = umeh.loc[gal_name, 'z']
    template = read_template(template)
    z_temp_wavel = template.rest_wavelength*(1+z)
    # Fetch useful quantities from table(s)
    gal_fluxes = umeh_flux.iloc[:,::2].loc[gal_name].values
    gal_unc = np.sqrt(umeh_flux.iloc[:,1::2].loc[gal_name].values**-1)
    target_wavels = wave[7:]
    # Chi^2 Calculation
    masks = np.array([mask_wave(z_temp_wavel, current_wave) for current_wave in target_wavels])
    chi = np.sum(np.array([((gal_fluxes[i] - float(template.luminosity[masks[i]])*factor)/gal_unc[i])**2 for i in range(4)]))/4
    
    return chi

In [69]:
factor1 = fit_template('J0901', 'Composite1')
print(factor1)

maskcheck--- 1 1.435e+23
5.57632527609291e-30


In [70]:

chi_squared('J0905', 'Composite1', factor1)

342438015.8801947

In [71]:
factors = [fit_template(gal, tem) for gal in umeh.index.values for tem in templates]

maskcheck--- 1 1.547e+23
maskcheck--- 1 2.602e+23
maskcheck--- 1 2.083e+23
maskcheck--- 1 1.769e+23
maskcheck--- 1 4.768e+23
maskcheck--- 1 6.961e+23
maskcheck--- 1 9.803e+23
maskcheck--- 1 9.879e+22
maskcheck--- 1 2.009e+23
maskcheck--- 1 2.006e+23
maskcheck--- 1 1.702e+23
maskcheck--- 1 1.776e+23
maskcheck--- 1 3.168e+23
maskcheck--- 1 2.48e+23
maskcheck--- 1 6.869e+23
maskcheck--- 1 4.746e+23
maskcheck--- 1 1.269e+24
maskcheck--- 1 7.263e+23
maskcheck--- 1 1.435e+23
maskcheck--- 1 2.494e+23
maskcheck--- 1 1.977e+23
maskcheck--- 1 2.028e+23
maskcheck--- 1 5.431e+23
maskcheck--- 1 1.074e+24
maskcheck--- 1 1.251e+24
maskcheck--- 1 8.656e+22
maskcheck--- 1 2.214e+23
maskcheck--- 1 1.703e+23
maskcheck--- 1 1.57e+23
maskcheck--- 1 1.71e+23
maskcheck--- 1 3.653e+23
maskcheck--- 2 2.944e+23
maskcheck--- 1 6.653e+23
maskcheck--- 1 5.433e+23
maskcheck--- 1 1.549e+24
maskcheck--- 1 8.71e+23
maskcheck--- 1 1.607e+23
maskcheck--- 1 2.654e+23
maskcheck--- 1 2.118e+23
maskcheck--- 1 1.585e+23
mask

In [72]:
factors

[4.492723449184212e-30,
 2.6711157478431886e-30,
 3.3366505885203916e-30,
 3.928910783430173e-30,
 1.457685229842277e-30,
 9.984547013199218e-31,
 7.089914491367924e-31,
 7.035371167008782e-30,
 3.4595535967585744e-30,
 3.4647274057268074e-30,
 4.0835741338942284e-30,
 3.913425211648635e-30,
 2.1938898913787802e-30,
 2.8025174096322484e-30,
 1.0118275114118467e-30,
 1.4644423042326118e-30,
 5.476944977059083e-31,
 9.569383417166427e-31,
 5.57632527609291e-30,
 3.208511135201815e-30,
 4.0475603293845865e-30,
 3.9457725696219566e-30,
 1.4733984111937631e-30,
 7.450676695710733e-31,
 6.3965042135837955e-31,
 9.244485641397098e-30,
 3.6142849011713314e-30,
 4.698782601992558e-30,
 5.0968323383396985e-30,
 4.679547819411303e-30,
 2.1905356614271356e-30,
 2.7180797456499073e-30,
 1.2027696935507781e-30,
 1.4728560226750095e-30,
 5.165930775463736e-31,
 9.187171953149629e-31,
 2.9483845260661205e-30,
 1.7852501632962532e-30,
 2.2370415171804797e-30,
 2.9893084753238208e-30,
 1.153653258677442

In [73]:
len(factors)

216

In [77]:
12*18

216

In [76]:
len(templates)

18